Проведение экспериментов по настройке модели

Загрузка очищенной выборки и её разбиение на тестовую и обучающую части

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#загрузка очищенной выборки
df = pd.read_pickle('/home/mainuser/intelligent_information_systems/data/car_clean_data.pkl')

In [2]:
#получим информацию о выборке
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 299 entries, 0 to 300
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Selling_Price           299 non-null    float32 
 1   Present_Price           299 non-null    float32 
 2   Driven_kms              299 non-null    int32   
 3   Fuel_Type               299 non-null    category
 4   Selling_type            299 non-null    category
 5   Transmission            299 non-null    category
 6   Year_Category           299 non-null    category
 7   Car_Frequency_Category  299 non-null    category
dtypes: category(5), float32(2), int32(1)
memory usage: 7.4 KB


In [3]:
df = df.rename(columns={'Selling_Price': 'target'})

In [4]:
df

,target,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Year_Category,Car_Frequency_Category
0,3.35,5.59,27000,Petrol,Dealer,Manual,2011-2015,Medium_Frequency_Brands
1,4.75,9.54,43000,Diesel,Dealer,Manual,2011-2015,Medium_Frequency_Brands
2,7.25,9.85,6900,Petrol,Dealer,Manual,2016-2018,High_Frequency_Brands
3,2.85,4.15,5200,Petrol,Dealer,Manual,2011-2015,Medium_Frequency_Brands
4,4.60,6.87,42450,Diesel,Dealer,Manual,2011-2015,Medium_Frequency_Brands
...,...,...,...,...,...,...,...,...
296,9.50,11.60,33988,Diesel,Dealer,Manual,2016-2018,High_Frequency_Brands
297,4.00,5.90,60000,Petrol,Dealer,Manual,2011-2015,High_Frequency_Brands
298,3.35,11.00,87934,Petrol,Dealer,Manual,2003-2010,High_Frequency_Brands
299,11.50,12.50,9000,Diesel,Dealer,Manual,2016-2018,High_Frequency_Brands


In [5]:
#разбиваем выборку на обучающую и тестовую в пропорции - 75% и 25% соответственно. 
#из тестовой выборки убираем цену продажи, так будем ее предсказывать
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.25, random_state=2)

In [6]:
#выведу первую строку тестовой выборки
X_train.iloc[0]

Present_Price                            0.787
Driven_kms                               15000
Fuel_Type                               Petrol
Selling_type                        Individual
Transmission                            Manual
Year_Category                        2011-2015
Car_Frequency_Category    Low_Frequency_Brands
Name: 146, dtype: object

Создание переменных, содержащих названия столбцов с числовыми и категориальными признаками

In [7]:
#категориальные признаки
cat_features = X_train.select_dtypes(include=['category','object']).columns.to_list()
cat_features

['Fuel_Type',
 'Selling_type',
 'Transmission',
 'Year_Category',
 'Car_Frequency_Category']

In [8]:
#числовые признаки
num_features = X_train.select_dtypes(include=['number']).columns.to_list()
num_features

['Present_Price', 'Driven_kms']

Создание pipeline обработки признаков и обучения модели. Для числовых признаков использование StandardScaler, для категориальных - OrdinalEncoder для задачи регрессии. В качестве модели - RandomForest

In [9]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#создаем преобразователи
s_scaler = StandardScaler()
l_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1) # для категориальных признаков
regressor = RandomForestRegressor(random_state=42)

#создаем ColumnTransformer для обработки столбцов
# Для удобной работы со столбцами
preprocessor = ColumnTransformer(
    transformers=[
        ('num', s_scaler, num_features),  # преобразования для числовых признаков
        ('cat', l_encoder, cat_features), # преобразования для категориальных признаков
    ],
remainder='drop' ) # Удаляем столбцы, которые не затронуты преобразования

In [10]:
#создаем pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('model', regressor)
])

Обучение baseline-модели и получение метрик качества на тестовой выборке. Для регрессии - mae, mape, mse
Задача регрессии - так как предсказание цены продажи

In [11]:
#обучаем модель
pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [12]:
#создаем тестовые данные для предсказания
model_params = {
    "Present_Price": 0.787,
    "Driven_kms": 15000,
    "Fuel_Type": "Petrol",
    "Selling_type": "Individual", 
    "Transmission": "Manual",
    "Year_Category": "2011-2015",
    "Car_Frequency_Category": "Low_Frequency_Brands"
}

df_pred = pd.DataFrame(model_params, index=[0])
prediction = pipeline.predict(df_pred)

In [13]:
print(f"Предсказанная цена автомобиля: {prediction[0]:.2f}")

Предсказанная цена автомобиля: 0.59


In [14]:
#делаем предсказания
predictions = pipeline.predict(X_test)

In [15]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
metrics = {}
metrics["mae"] = mean_absolute_error(y_test, predictions)   
metrics["mape"] = mean_absolute_percentage_error(y_test, predictions)
metrics["mse"] = mean_squared_error(y_test, predictions)

metrics

{'mae': 0.7010266769975424,
 'mape': 0.20053071605152803,
 'mse': 3.8858707894773867}

Настройка MLFlow и логирование baseline-модели

In [16]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# Настройка MLFlow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("car_price_prediction")

<Experiment: artifact_location='/home/mainuser/intelligent_information_systems/mlflow/mlartifacts/1', creation_time=1761293424108, experiment_id='1', last_update_time=1761293424108, lifecycle_stage='active', name='car_price_prediction', tags={}>

In [17]:
# Подготовка артефактов для логирования
signature = infer_signature(X_train.head(5), pipeline.predict(X_train.head(5)))
input_example = X_train.head(5)

# Логирование baseline модели
RUN_NAME = "baseline_rf"

with mlflow.start_run(run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    
    mlflow.sklearn.log_model(
        pipeline,
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        registered_model_name="car_price_model"
    )
    
    mlflow.log_metrics(metrics)
    mlflow.log_params(pipeline.get_params())
    
    # Логируем список признаков
    with open("features.txt", "w") as f:
        f.write("\n".join(num_features + cat_features))
    mlflow.log_artifact("features.txt")

print(f"Run completed: {run_id}")

/home/mainuser/intelligent_information_systems/.venv_my_proj/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/06 00:04:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'car_price_model' already exists. Creating a new version of this model...


🏃 View run baseline_rf at: http://localhost:5000/#/experiments/1/runs/b66966880a174126ac4004b3eeadf57e
🧪 View experiment at: http://localhost:5000/#/experiments/1
Run completed: b66966880a174126ac4004b3eeadf57e


Создание дополнительных признаков с sklearn

In [18]:
from sklearn.preprocessing import PolynomialFeatures, KBinsDiscretizer
from sklearn.pipeline import FeatureUnion

# Создаем копию данных
X_train_fe_sklearn = X_train.copy()

# Создаем дополнительные трансформации
poly_features = PolynomialFeatures(degree=2, include_bias=False)
kbin_features = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')

# Расширенный ColumnTransformer с новыми признаками
extended_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
        ('poly', poly_features, ['Present_Price', 'Driven_kms']),
        ('kbin', kbin_features, ['Present_Price', 'Driven_kms'])
    ],
    remainder='drop'
)

# Обучаем и преобразуем данные
X_train_fe_sklearn_transformed = extended_preprocessor.fit_transform(X_train_fe_sklearn)

# Сохраняем названия столбцов
feature_names = extended_preprocessor.get_feature_names_out()
with open('sklearn_features.txt', 'w') as f:
    f.write('\n'.join(feature_names))

# Создаем новый pipeline с расширенными признаками
pipeline_sklearn_fe = Pipeline(steps=[
    ('preprocessor', extended_preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

# Обучаем модель
pipeline_sklearn_fe.fit(X_train, y_train)

# Оцениваем качество
predictions_sklearn_fe = pipeline_sklearn_fe.predict(X_test)
metrics_sklearn_fe = {
    "mae": mean_absolute_error(y_test, predictions_sklearn_fe),
    "mape": mean_absolute_percentage_error(y_test, predictions_sklearn_fe),
    "mse": mean_squared_error(y_test, predictions_sklearn_fe)
}

print("Metrics with sklearn features:", metrics_sklearn_fe)

/home/mainuser/intelligent_information_systems/.venv_my_proj/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/home/mainuser/intelligent_information_systems/.venv_my_proj/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


Metrics with sklearn features: {'mae': 0.730522894022531, 'mape': 0.20904083348231797, 'mse': 4.583147402775556}


In [19]:
# Логирование модели с sklearn features
RUN_NAME = "sklearn_feature_engineering"

with mlflow.start_run(run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    
    mlflow.sklearn.log_model(
        pipeline_sklearn_fe,
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        registered_model_name="car_price_model"
    )
    
    mlflow.log_metrics(metrics_sklearn_fe)
    mlflow.log_params(pipeline_sklearn_fe.get_params())
    mlflow.log_artifact("sklearn_features.txt")
    
    # Сохраняем информацию о количестве признаков
    mlflow.log_param("num_features_after_fe", len(feature_names))

print(f"Sklearn FE Run completed: {run_id}")

2025/11/06 00:04:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'car_price_model' already exists. Creating a new version of this model...
2025/11/06 00:05:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_model, version 11
Created version '11' of model 'car_price_model'.


🏃 View run sklearn_feature_engineering at: http://localhost:5000/#/experiments/1/runs/7b4670f45c1e428c83742277474b1dd0
🧪 View experiment at: http://localhost:5000/#/experiments/1
Sklearn FE Run completed: 7b4670f45c1e428c83742277474b1dd0


Генерация признаков с autofeat

In [23]:
!pip install autofeat

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


In [20]:
# Решение: используем autofeat без многопоточности и с другими настройками
from autofeat import AutoFeatRegressor

# Создаем копию данных
X_train_autofeat = X_train.copy()

# Инициализируем AutoFeatRegressor с отключенной многопоточностью
af_regressor = AutoFeatRegressor(
    feateng_steps=1,  # уменьшаем для скорости
    verbose=1,
    n_jobs=1,  # отключаем многопоточность
    featsel_runs=5   # уменьшаем количество прогонов
)

# Применяем autofeat только к числовым признакам
print("Applying autofeat transformations...")
try:
    X_train_autofeat_transformed = af_regressor.fit_transform(X_train_autofeat[num_features], y_train)
    
    # Объединяем с категориальными признаками
    X_train_autofeat_full = pd.concat([
        X_train_autofeat_transformed.reset_index(drop=True),
        X_train_autofeat[cat_features].reset_index(drop=True)
    ], axis=1)

    print(f"Original features: {len(num_features + cat_features)}")
    print(f"After autofeat: {X_train_autofeat_full.shape[1]}")
    
    # Сохраняем названия autofeat признаков
    autofeat_feature_names = X_train_autofeat_full.columns.tolist()
    with open('autofeat_features.txt', 'w') as f:
        f.write('\n'.join(autofeat_feature_names))
        
except Exception as e:
    print(f"Autofeat failed: {e}")
    print("Skipping autofeat and moving to next step")

2025-11-06 00:06:51,815 INFO: [AutoFeat] The 1 step feature engineering process could generate up to 14 features.
2025-11-06 00:06:51,817 INFO: [AutoFeat] With 224 data points this new feature matrix would use about 0.00 gb of space.
2025-11-06 00:06:51,819 INFO: [feateng] Step 1: transformation of original features


Applying autofeat transformations...


2025-11-06 00:06:57,859 INFO: [feateng] Generated 9 transformed features from 2 original features - done.
2025-11-06 00:06:59,360 INFO: [feateng] Generated altogether 9 new features in 1 steps
2025-11-06 00:06:59,362 INFO: [feateng] Removing correlated features, as well as additions at the highest level
2025-11-06 00:06:59,385 INFO: [feateng] Generated a total of 8 additional features
2025-11-06 00:06:59,444 INFO: [featsel] Feature selection run 1/5


[featsel] Scaling data...done.


2025-11-06 00:06:59,675 INFO: [featsel] Feature selection run 2/5
2025-11-06 00:06:59,811 INFO: [featsel] Feature selection run 3/5
2025-11-06 00:06:59,947 INFO: [featsel] Feature selection run 4/5
2025-11-06 00:07:00,113 INFO: [featsel] Feature selection run 5/5
2025-11-06 00:07:00,235 INFO: [featsel] 8 features after 5 feature selection runs
/home/mainuser/intelligent_information_systems/.venv_my_proj/lib/python3.10/site-packages/autofeat/featsel.py:270: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  if np.max(np.abs(correlations[c].ravel()[:i])) < 0.9:
2025-11-06 00:07:00,245 INFO: [featsel] 7 features after correlation filtering
2025-11-06 00:07:00,290 INFO: [featsel] 6 features after noise filtering
2025-11-06 00:07:00,294 INFO: [AutoFeat] Computing 4 new features.


2025-11-06 00:07:01,764 INFO: [AutoFeat]     4/    4 new features ...done.
2025-11-06 00:07:01,769 INFO: [AutoFeat] Final dataframe with 6 feature columns (4 new).
2025-11-06 00:07:01,770 INFO: [AutoFeat] Training final regression model.
2025-11-06 00:07:01,796 INFO: [AutoFeat] Trained model: largest coefficients:
2025-11-06 00:07:01,798 INFO: 2.15314049791765
2025-11-06 00:07:01,802 INFO: 0.600143 * Present_Price
2025-11-06 00:07:01,805 INFO: -0.000066 * Driven_kms
2025-11-06 00:07:01,806 INFO: 0.000047 * Present_Price**3
2025-11-06 00:07:01,812 INFO: [AutoFeat] Final score: 0.8388


Original features: 7
After autofeat: 11


In [21]:
# Создаем и обучаем пайплайн для autofeat с обработкой категориальных признаков
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

# Определяем категориальные колонки в autofeat данных
cat_cols_autofeat = X_train_autofeat_full.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols_autofeat = X_train_autofeat_full.select_dtypes(include=['number']).columns.tolist()

# Создаем препроцессор для autofeat данных
preprocessor_autofeat = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_cols_autofeat)
    ],
    remainder='passthrough'  # числовые признаки оставляем как есть
)

pipeline_autofeat = Pipeline(steps=[
    ('preprocessor', preprocessor_autofeat),
    ('model', RandomForestRegressor(random_state=42))
])

# Обучаем на autofeat данных
pipeline_autofeat.fit(X_train_autofeat_full, y_train)
print("Autofeat pipeline trained successfully")

Autofeat pipeline trained successfully


In [22]:
# Применяем autofeat к тестовым данным
X_test_autofeat_transformed = af_regressor.transform(X_test[num_features])

# Объединяем с категориальными признаками для теста
X_test_autofeat_full = pd.concat([
    X_test_autofeat_transformed.reset_index(drop=True),
    X_test[cat_features].reset_index(drop=True)
], axis=1)

# Оцениваем качество
predictions_autofeat = pipeline_autofeat.predict(X_test_autofeat_full)
metrics_autofeat = {
    "mae": mean_absolute_error(y_test, predictions_autofeat),
    "mape": mean_absolute_percentage_error(y_test, predictions_autofeat),
    "mse": mean_squared_error(y_test, predictions_autofeat)
}

print("Metrics with autofeat features:", metrics_autofeat)

# Логируем модель с autofeat
RUN_NAME = "autofeat_features"

with mlflow.start_run(run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    
    mlflow.sklearn.log_model(
        pipeline_autofeat,
        artifact_path="models",
        signature=infer_signature(X_train_autofeat_full.head(5), 
                                pipeline_autofeat.predict(X_train_autofeat_full.head(5))),
        input_example=X_train_autofeat_full.head(5),
        registered_model_name="car_price_model"
    )
    
    mlflow.log_metrics(metrics_autofeat)
    mlflow.log_params(pipeline_autofeat.get_params())
    mlflow.log_artifact("autofeat_features.txt")
    mlflow.log_param("num_autofeat_features", X_train_autofeat_full.shape[1])

print(f"Autofeat Run completed: {run_id}")

/home/mainuser/intelligent_information_systems/.venv_my_proj/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-11-06 00:07:13,699 INFO: [AutoFeat] Computing 4 new features.
2025-11-06 00:07:13,709 INFO: [AutoFeat]     4/    4 new features ...done.


Metrics with autofeat features: {'mae': 0.7134721223347716, 'mape': 0.2044283264837233, 'mse': 4.094777230435301}


2025/11/06 00:07:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'car_price_model' already exists. Creating a new version of this model...
2025/11/06 00:07:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_model, version 12
Created version '12' of model 'car_price_model'.


🏃 View run autofeat_features at: http://localhost:5000/#/experiments/1/runs/3df153eb5e764582b718c70eff0131c7
🧪 View experiment at: http://localhost:5000/#/experiments/1
Autofeat Run completed: 3df153eb5e764582b718c70eff0131c7


Отбор признаков с mlxtend

In [33]:
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.8 MB/s eta 0:00:0000:0100:010m


In [23]:
from mlxtend.feature_selection import SequentialFeatureSelector

# Используем признаки из sklearn feature engineering
X_train_sklearn_features = X_train_fe_sklearn_transformed

# Определяем количество признаков для отбора (20-70% от общего числа)
n_features = len(feature_names)
n_select = int(n_features * 0.4)  # 40% признаков
print(f"Total features: {n_features}, selecting: {n_select}")

# Создаем SequentialFeatureSelector
sfs = SequentialFeatureSelector(
    RandomForestRegressor(n_estimators=10, random_state=42),  # упрощенная модель для скорости
    k_features=n_select,
    forward=True,
    floating=False,
    scoring='neg_mean_absolute_error',
    cv=3,
    n_jobs=1
)

# Обучаем селектор
print("Training feature selector...")
sfs.fit(X_train_sklearn_features, y_train)

# Получаем отобранные признаки
selected_idx = list(sfs.k_feature_idx_)
selected_features = [feature_names[i] for i in selected_idx]

print(f"Selected {len(selected_features)} features:")
for feat in selected_features:
    print(f"  - {feat}")

# Сохраняем информацию об отобранных признаках
with open('sfs_selected_idx.txt', 'w') as f:
    f.write(str(selected_idx))
with open('sfs_selected_features.txt', 'w') as f:
    f.write('\n'.join(selected_features))

Total features: 14, selecting: 5
Training feature selector...
Selected 5 features:
  - num__Present_Price
  - cat__Fuel_Type
  - cat__Year_Category
  - poly__Present_Price Driven_kms
  - poly__Driven_kms^2


SFS отобрал 5 наиболее важных признаков.
Теперь создадим и обучим модель с отобранными признаками.

In [24]:
# Создаем ColumnExtractor для отбора признаков
class ColumnExtractor:
    def __init__(self, indices):
        self.indices = indices
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        return X[:, self.indices]

# Создаем пайплайн с отбором признаков
pipeline_sfs = Pipeline(steps=[
    ('preprocessor', extended_preprocessor),  # тот же препроцессор что и для sklearn features
    ('feature_selector', ColumnExtractor(selected_idx)),
    ('model', RandomForestRegressor(random_state=42))
])

# Обучаем модель
pipeline_sfs.fit(X_train, y_train)

# Оцениваем качество
predictions_sfs = pipeline_sfs.predict(X_test)
metrics_sfs = {
    "mae": mean_absolute_error(y_test, predictions_sfs),
    "mape": mean_absolute_percentage_error(y_test, predictions_sfs),
    "mse": mean_squared_error(y_test, predictions_sfs)
}

print("Metrics with SFS features:", metrics_sfs)

# Логируем модель с отобранными признаками
RUN_NAME = "sfs_feature_selection"

with mlflow.start_run(run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    
    mlflow.sklearn.log_model(
        pipeline_sfs,
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        registered_model_name="car_price_model"
    )
    
    mlflow.log_metrics(metrics_sfs)
    mlflow.log_params(pipeline_sfs.get_params())
    mlflow.log_artifact("sfs_selected_idx.txt")
    mlflow.log_artifact("sfs_selected_features.txt")
    mlflow.log_param("num_sfs_features", len(selected_features))

print(f"SFS Run completed: {run_id}")

/home/mainuser/intelligent_information_systems/.venv_my_proj/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
2025/11/06 00:07:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Metrics with SFS features: {'mae': 0.7400060073683657, 'mape': 0.2095523735694632, 'mse': 5.118186845668666}


2025-11-06 00:08:09,152 WARNING: Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /api/2.0/mlflow/registered-models/create
Registered model 'car_price_model' already exists. Creating a new version of this model...
2025/11/06 00:08:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_model, version 13
Created version '13' of model 'car_price_model'.


🏃 View run sfs_feature_selection at: http://localhost:5000/#/experiments/1/runs/ba2560df923a47c4b71f32c439041235
🧪 View experiment at: http://localhost:5000/#/experiments/1
SFS Run completed: ba2560df923a47c4b71f32c439041235


Дополнительные методы отбора признаков. RFE (Recursive Feature Elimination)

In [25]:
from sklearn.feature_selection import RFE

In [26]:
# Создаем RFE селектор
rfe_selector = RFE(
    estimator=RandomForestRegressor(n_estimators=10, random_state=42),
    n_features_to_select=6,  # выбираем 6 признаков
    step=1
)

In [27]:
print("Training RFE selector...")
rfe_selector.fit(X_train_sklearn_features, y_train)

Training RFE selector...


,estimator,RandomForestR...ndom_state=42)
,n_features_to_select,6
,step,1
,verbose,0
,importance_getter,'auto'
,n_estimators,10
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0


In [28]:
# Получаем отобранные признаки
rfe_selected_idx = rfe_selector.support_
rfe_selected_features = [feature_names[i] for i, selected in enumerate(rfe_selected_idx) if selected]

In [29]:
print(f"RFE selected {len(rfe_selected_features)} features:")
for feat in rfe_selected_features:
    print(f"  - {feat}")

RFE selected 6 features:
  - num__Present_Price
  - num__Driven_kms
  - cat__Year_Category
  - poly__Present_Price
  - poly__Present_Price^2
  - poly__Driven_kms^2


RFE отобрал 6 наиболее важных признаков.

In [30]:
# Сохраняем информацию об отобранных признаках
with open('rfe_selected_idx.txt', 'w') as f:
    f.write(str(rfe_selected_idx.tolist()))
with open('rfe_selected_features.txt', 'w') as f:
    f.write('\n'.join(rfe_selected_features))

In [31]:
# Сравниваем с SFS
print("\nComparison with SFS:")
print(f"SFS features: {set(selected_features)}")
print(f"RFE features: {set(rfe_selected_features)}")
print(f"Common features: {set(selected_features) & set(rfe_selected_features)}")


Comparison with SFS:
SFS features: {'cat__Year_Category', 'poly__Driven_kms^2', 'cat__Fuel_Type', 'poly__Present_Price Driven_kms', 'num__Present_Price'}
RFE features: {'num__Driven_kms', 'cat__Year_Category', 'poly__Driven_kms^2', 'poly__Present_Price', 'poly__Present_Price^2', 'num__Present_Price'}
Common features: {'num__Present_Price', 'cat__Year_Category', 'poly__Driven_kms^2'}


Настройка гиперпараметров с Optuna

In [39]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 KB 2.5 MB/s eta 0:00:0000:0100:01


In [32]:
import optuna
from sklearn.metrics import mean_absolute_error

def objective(trial):
    # Параметры для оптимизации
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
    }
    
    # Используем лучший пайплайн (например, с sklearn features)
    model = RandomForestRegressor(**params, random_state=42)
    pipeline_optuna = Pipeline(steps=[
        ('preprocessor', extended_preprocessor),
        ('model', model)
    ])
    
    # Обучаем и оцениваем
    pipeline_optuna.fit(X_train, y_train)
    predictions = pipeline_optuna.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    
    return mae

# Запускаем оптимизацию
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2025-11-06 00:10:28,026] A new study created in memory with name: no-name-7db1e8b4-0957-45d7-bc45-2056678cd63e
/home/mainuser/intelligent_information_systems/.venv_my_proj/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
[I 2025-11-06 00:10:28,511] Trial 0 finished with value: 0.7803391453278599 and parameters: {'n_estimators': 73, 'max_depth': 13, 'max_features': 0.6546138062940123, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7803391453278599.
/home/mainuser/intelligent_information_systems/.venv_my_proj/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current

Обучу модель с лучшими параметрами и залогирую в MLFlow

In [33]:
# Получаем лучшие параметры
best_params = study.best_params
print("Best trial:")
print(f"MAE: {study.best_value}")
print(f"Best hyperparameters: {best_params}")

# Обучаем финальную модель с лучшими параметрами
best_model = RandomForestRegressor(**best_params, random_state=42)
pipeline_optuna = Pipeline(steps=[
    ('preprocessor', extended_preprocessor),
    ('model', best_model)
])

# Обучаем на всех тренировочных данных
pipeline_optuna.fit(X_train, y_train)

# Оцениваем качество
predictions_optuna = pipeline_optuna.predict(X_test)
metrics_optuna = {
    "mae": mean_absolute_error(y_test, predictions_optuna),
    "mape": mean_absolute_percentage_error(y_test, predictions_optuna),
    "mse": mean_squared_error(y_test, predictions_optuna)
}

print("Metrics with optimized hyperparameters:", metrics_optuna)

# Логируем оптимизированную модель
RUN_NAME = "optuna_optimized"

with mlflow.start_run(run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    
    mlflow.sklearn.log_model(
        pipeline_optuna,
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        registered_model_name="car_price_model"
    )
    
    mlflow.log_metrics(metrics_optuna)
    mlflow.log_params(best_params)
    mlflow.log_param("optimization_method", "optuna")
    mlflow.log_param("num_trials", 20)

print(f"Optuna Run completed: {run_id}")

Best trial:
MAE: 0.737107257339109
Best hyperparameters: {'n_estimators': 210, 'max_depth': 15, 'max_features': 0.9140683736816175, 'min_samples_split': 4, 'min_samples_leaf': 2}


/home/mainuser/intelligent_information_systems/.venv_my_proj/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
2025/11/06 00:10:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Metrics with optimized hyperparameters: {'mae': 0.737107257339109, 'mape': 0.21200978650584096, 'mse': 4.735284491742066}


Registered model 'car_price_model' already exists. Creating a new version of this model...
2025/11/06 00:11:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_model, version 14
Created version '14' of model 'car_price_model'.


🏃 View run optuna_optimized at: http://localhost:5000/#/experiments/1/runs/6795beeba25c4c3d9fc72639385a9b49
🧪 View experiment at: http://localhost:5000/#/experiments/1
Optuna Run completed: 6795beeba25c4c3d9fc72639385a9b49


In [35]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 KB 6.5 MB/s eta 0:00:00


In [37]:
from catboost import CatBoostRegressor

# Для CatBoost создаем отдельный препроцессор без кодирования категорий (CatBoost сам умеет)
preprocessor_catboost = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
    ],
    remainder='passthrough'  # категориальные признаки оставляем как есть
)

# Обучаем CatBoost на данных с категориальными признаками
X_train_processed = preprocessor_catboost.fit_transform(X_train)
X_test_processed = preprocessor_catboost.transform(X_test)

# CatBoost модель
catboost_model = CatBoostRegressor(
    random_state=42,
    verbose=False,
    iterations=100,
    cat_features=[X_train_processed.shape[1] - len(cat_features) + i for i in range(len(cat_features))]  # указываем индексы категориальных признаков
)

# Обучаем
catboost_model.fit(X_train_processed, y_train)

# Оцениваем
predictions_catboost = catboost_model.predict(X_test_processed)
metrics_catboost = {
    "mae": mean_absolute_error(y_test, predictions_catboost),
    "mape": mean_absolute_percentage_error(y_test, predictions_catboost),
    "mse": mean_squared_error(y_test, predictions_catboost)
}

print("Metrics with CatBoost:", metrics_catboost)

# Логируем CatBoost модель
RUN_NAME = "catboost_model"

with mlflow.start_run(run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    
    mlflow.catboost.log_model(
        catboost_model,
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        registered_model_name="car_price_model"
    )
    
    mlflow.log_metrics(metrics_catboost)
    mlflow.log_params(catboost_model.get_params())

print(f"CatBoost Run completed: {run_id}")

Metrics with CatBoost: {'mae': 0.7919420178049162, 'mape': 0.31984289972034785, 'mse': 6.034655756300794}


2025/11/06 00:15:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-11-06 00:16:02,019 WARNING: Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /api/2.0/mlflow/logged-models/m-5e93f4e9d26c47848f22493b84339985
Registered model 'car_price_model' already exists. Creating a new version of this model...
2025/11/06 00:16:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_model, version 15
Created version '15' of model 'car_price_model'.


🏃 View run catboost_model at: http://localhost:5000/#/experiments/1/runs/bb7232b524774b72ad6a62a690729a8d
🧪 View experiment at: http://localhost:5000/#/experiments/1
CatBoost Run completed: bb7232b524774b72ad6a62a690729a8d


In [38]:
# сравниваем MAE всех моделей
models_mae = {
    'Baseline': metrics['mae'],
    'Sklearn FE': metrics_sklearn_fe['mae'],
    'Autofeat': metrics_autofeat['mae'], 
    'SFS': metrics_sfs['mae'],
    'Optuna': metrics_optuna['mae'],
    'CatBoost': metrics_catboost['mae']
}

# Сортируем от лучшей к худшей
sorted_models = sorted(models_mae.items(), key=lambda x: x[1])

for i, (name, mae) in enumerate(sorted_models, 1):
    print(f"{i}. {name}: MAE = {mae:.3f}")

# Выбираем лучшую
best_name, best_mae = sorted_models[0]
print(best_name, best_mae)

# Определяем какой пайплайн использовать
if best_name == 'Optuna':
    best_pipeline = pipeline_optuna
elif best_name == 'Sklearn FE':
    best_pipeline = pipeline_sklearn_fe
elif best_name == 'Autofeat':
    best_pipeline = pipeline_autofeat
elif best_name == 'SFS':
    best_pipeline = pipeline_sfs
elif best_name == 'CatBoost':
    best_pipeline = catboost_model
else:
    best_pipeline = pipeline

print(f"Используем пайплайн: {best_name}")

1. Baseline: MAE = 0.701
2. Autofeat: MAE = 0.713
3. Sklearn FE: MAE = 0.731
4. Optuna: MAE = 0.737
5. SFS: MAE = 0.740
6. CatBoost: MAE = 0.792
Baseline 0.7010266769975424
Используем пайплайн: Baseline


In [43]:
# Создаем новый корректный Production run
RUN_NAME = "production_final"

with mlflow.start_run(run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    
    # Логируем модель - ВАЖНО: используем правильные параметры
    mlflow.sklearn.log_model(
        sk_model=production_pipeline,
        artifact_path="model",  # должно быть "model" а не "models"
        registered_model_name="car_price_model"
    )
    
    print(f"Модель залогирована в run: {run_id}")
    
    # Проверим что артефакты создались
    client = mlflow.tracking.MlflowClient()
    artifacts = client.list_artifacts(run_id)
    print("Артефакты после логирования:")
    for art in artifacts:
        print(f"  - {art.path}")

print("Production модель перелогирована!")

# Установим эту версию как Production
versions = client.search_model_versions("name='car_price_model'")
new_version = max([v.version for v in versions])
print(f"Новая версия: {new_version}")

client.transition_model_version_stage(
    name="car_price_model", 
    version=new_version,
    stage="Production"
)

print(f"Версия {new_version} установлена как Production!")

2025/11/06 00:58:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/06 00:58:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'car_price_model' already exists. Creating a new version of this model...
2025/11/06 00:59:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_model, version 18
Created version '18' of model 'car_price_model'.


Модель залогирована в run: 4a08cd3868eb463baaee2dcbf614a72c
Артефакты после логирования:
🏃 View run production_final at: http://localhost:5000/#/experiments/1/runs/4a08cd3868eb463baaee2dcbf614a72c
🧪 View experiment at: http://localhost:5000/#/experiments/1
Production модель перелогирована!
Новая версия: 9


/tmp/ipykernel_5255/870600265.py:30: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Версия 9 установлена как Production!


In [44]:
# Проверим текущую Production версию
client = mlflow.tracking.MlflowClient()
versions = client.search_model_versions("name='car_price_model'")

print("Все версии модели car_price_model:")
for v in versions:
    stage = v.current_stage if v.current_stage else "None"
    print(f"Version {v.version}: {stage} - Run ID: {v.run_id}")

# Найдем Production версию
prod_versions = [v for v in versions if v.current_stage == "Production"]
if prod_versions:
    prod_version = prod_versions[0]
    print(f"\n✅ Production версия: {prod_version.version}")
    print(f"Run ID: {prod_version.run_id}")
else:
    print("\n❌ Нет Production версии!")

Все версии модели car_price_model:
Version 9: Production - Run ID: a62f28ce8151419c91546e5ab24f8ec6
Version 18: None - Run ID: 4a08cd3868eb463baaee2dcbf614a72c
Version 17: Production - Run ID: 19c20a4b16c149c2adcc3e2418fd4a98
Version 16: Archived - Run ID: cfdbdf386a684b2191be53c6b52bfaf6
Version 15: None - Run ID: bb7232b524774b72ad6a62a690729a8d
Version 14: None - Run ID: 6795beeba25c4c3d9fc72639385a9b49
Version 13: None - Run ID: ba2560df923a47c4b71f32c439041235
Version 12: None - Run ID: 3df153eb5e764582b718c70eff0131c7
Version 11: None - Run ID: 7b4670f45c1e428c83742277474b1dd0
Version 10: None - Run ID: b66966880a174126ac4004b3eeadf57e
Version 8: None - Run ID: 752cf173afcf484a8f8e0f9af70dba3f
Version 7: None - Run ID: d1d467fb6599494aa9ed18240f3d35b6
Version 6: None - Run ID: 17f2477c02e746898a771d46991791bb
Version 5: None - Run ID: 335b4089c13a433d8d6b5f1b1fb6f9af
Version 4: None - Run ID: b3e7d10f5fa740ccbbb8281318683029
Version 3: None - Run ID: a8de5758bdb743f08e002afc65e66